## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [2]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


#### Look for subjects who do not have EO or EC cropped data

In [7]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

[]
[]


### Compute STCs

In [11]:
nan_subjects=[]

methods = [
    'MNE',
    # 'dSPM',
    ]

for sub_id in sub_ids: 
    # Check if eyes open or eyes closed need to be computed
    EO_bool = False if sub_id in no_eyes_open else True
    if not EO_bool:
        print(f"Subject: {sub_id} missing eyes open data")
        break
    
    for save_stc_mat in [
        False,
        # True,
        ]:    
        for method in methods:
            print(f"Source Localization Method: {method}")                          

            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method}) 046')
                           
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            # zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = True

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = sl.to_source(sub_id,
                                                             processed_data_path,
                                                             zscored_epochs_save_path,
                                                             EC_resting_save_path,
                                                             EO_resting_save_path,
                                                             roi_names,
                                                             times_tup,
                                                             method=method,
                                                             return_zepochs= Epochs_bool,
                                                             return_EC_resting = False,
                                                             return_EO_resting = EO_bool,
                                                             average_dipoles=average_dipoles,
                                                             save_stc_mat=save_stc_mat,
                                                             save_inv=save_inv,)
            utils.clear_display()
            
            # add nan subjects to list if not None
            if sub_id_if_nan is not None:
                nan_subjects.append(sub_id_if_nan)

Saved 046_inv.pkl to ../../Data/Source Time Courses (MNE) 046/zscored_Epochs/5_sec_time_window.


In [ ]:
nan_subjects